# (실습-10) RAG 실습

##실습 개요
1) 실습 목적 <br>
  이번 실습에서는 ColBERT와 LLM을 이용하여 RAG를 구현해 본다. <br>
  RAG 구현을 위해 LLM 프롬프트를 어떻게 사용할 수 있는지 확인한다. <br>
2) 수강 목표
  * RAG 구현을 위한 내부 모듈 및 시스템 파이프라인을 이해한다.
  * ColBERT와 OpenAI API를 사용하여 RAG를 실제로 구현할 수 있다.
  * Function calling을 포함한 프롬프트 엔지니어링 방법에 대해 이해한다.

### 실습 목차
* 1. RAG 개념 이해
* 2. 검색엔진 준비 - ColBERT
* 3. 대화형 IR

### 데이터셋 개요
* 데이터셋: LoTTE 벤치마크의 dev 세트
* 데이터셋 개요
  * LoTTE: ColBERT에서 소개하는 공개 데이터 셋이다.
* 데이터셋 라이센스
  * LoTTE: MIT
* LoTTE 데이터셋 상세 설명 <br>
 LoTTE 데이터 세트는 Wikipedia와 같은 엔티티 중심의 지식 기반에서는 잘 다루지 않을 수 있는 롱테일 주제에 대한 IR 시스템을 평가하기 위해 설계되었습니다. 이번 실습에서는 주로 반려 동물을 다루고 있는 문서를 사용합니다.

### 환경 설정
OpenAI 패키지를 설치한다. <br>
RAG용 검색엔진으로 사용할 ColBERT 패키지를 설치한다.

In [78]:
# OpenAI Python 패키지 설치
!pip install openai

# 검색엔진을 위해 ColBERT 사용
!pip install "colbert-ir[faiss-gpu, torch]"

In [79]:
from datasets import load_dataset

# LoTTE dataset 사용

dataset = 'lifestyle'
datasplit = 'dev'

collection_dataset = load_dataset("colbertv2/lotte_passages", dataset)
collection = [x['text'] for x in collection_dataset[datasplit + '_collection']]

queries_dataset = load_dataset("colbertv2/lotte", dataset)
queries = [x['query'] for x in queries_dataset['search_' + datasplit]]

print(f'Loaded {len(queries)} queries and {len(collection):,} passages')
print(queries[0])
print(collection[0])

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for colbertv2/lotte_passages contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/colbertv2/lotte_passages
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loaded 417 queries and 268,893 passages
how much should i feed my 1 year old english mastiff?
In my experience rabbits are very easy to housebreak. They like to pee and poop in the same place every time, so in most cases all you have to do is put a little bit of their waste in the litter box and they will happily use the litter box. It is very important that if they go somewhere else, miss the edge or kick waste out of the box that you clean it up well and immediately as otherwise those spots will become existing places to pee and poop. When you clean the box, save a little bit of waste and put it in the cleaned box so it smells right to them. For a more foolproof method, you can get a piece of wood soaked with their urine and put that in the box along with droppings or cage them so that they are only in their litter box for a week. Generally, if I try the first method and find that they are not using only the box on the first day, I go for the litter box only for a week method. The wo

## 1. RAG 개념 이해

레퍼런스 정보가 있다고 가정하고 그 정보를 토대로 질문에 답하는 예시

In [80]:
# OpenAI 라이브러리 및 필요한 모듈을 가져옵니다.
import os
import json
from openai import OpenAI

# OpenAI API 키를 환경변수에서 설정합니다.
os.environ["OPENAI_API_KEY"] = ""

client = OpenAI()
# 사용할 모델을 설정합니다. 여기서는 gpt-3.5-turbo-1106 모델을 사용합니다.
#llm_model = "gpt-4-1106-preview"
llm_model = "gpt-3.5-turbo-1106"

In [72]:
messages = [
{"role": "user", "content": "너는 누구니"},
{"role": "assistant", "content": "저는 식당 검색 도우미 Good Place 라고 합니다."},
{"role": "user", "content": "강남역 근처 태국음식 추천해줘"}
]

reference = [
{"식당명": "파파야리프", "대표 메뉴": "똠양꿍", "분위기": "이국적/이색적", "기타": "데이트에 적합"},
{"식당명": "할랄가이즈", "대표 메뉴": "할랄", "분위기": "무슬림, 이국적", "기타": "가성비"},
{"식당명": "인더비엣", "대표 메뉴": "팟타이, 쌀국수", "분위기": "깔끔", "기타": "해장"}
]

persona = """
## Name: Good Place

## Role: 식당 검색 도우미

## Instruction
- 사용자의 이전 메시지 정보 및 주어진 검색 결과(JSON 형태로 제공) 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 사용자가 사용한 언어로 답변을 생성한다.
"""

In [81]:
# 레퍼런스 없이 답변 생성하는 경우 - hallucination 발생

result = client.chat.completions.create(
    model=llm_model,
    messages=messages,
    temperature=0,
    seed=1
)

print(result.choices[0].message)

ChatCompletionMessage(content='강남역 근처에서 태국음식을 즐길 수 있는 몇 가지 좋은 식당을 추천해 드릴게요. \n\n1. 태국촌: 강남역 11번 출구에서 도보로 약 5분 거리에 위치한 이 식당은 정통 태국 음식을 맛볼 수 있습니다. 특히, 태국촌의 톰양껍데기와 그린 커리가 매우 유명합니다.\n\n2. 태국피플: 강남역 10번 출구에서 도보로 약 7분 거리에 위치한 이 식당은 분위기 좋은 인테리어와 맛있는 태국 음식으로 유명합니다. 특히, 태국피플의 팟타이와 쌀국수가 매우 인기가 있어요.\n\n이런 식당들 중에서 선택하시면 좋을 것 같습니다. 맛있는 식사 되시길 바라요!', role='assistant', function_call=None, tool_calls=None)


In [82]:
# 레퍼런스를 토대로 답변 생성하게 하는 경우


content = {
    "이전 메시지 정보": messages,
    "검색 결과": reference
}

msg = [{"role": "system", "content": persona}, {"role": "assistant",  "content": json.dumps(content, ensure_ascii=False, indent=4)}]

result = client.chat.completions.create(
    model=llm_model,
    messages=msg,
    temperature=0,
    seed=1
)

print(result.choices[0].message)

ChatCompletionMessage(content='강남역 근처 태국음식 추천해드릴게요. 파파야리프는 똠양꿍이 대표 메뉴로 이국적이고 데이트에 적합한 분위기를 가지고 있어요. 또한, 할랄가이즈는 할랄 메뉴로 가성비가 좋고 이국적인 분위기를 가지고 있어요. 마지막으로, 인더비엣은 팟타이와 쌀국수가 대표 메뉴로 깔끔한 분위기와 해장 메뉴로 유명해요. 이 중에서 원하시는 곳이 있나요?', role='assistant', function_call=None, tool_calls=None)


## 2. 검색엔진 준비 - ColBERT




빠른 실습을 위해 처음 10,000개의 구절에 대해서만 색인

In [83]:
# 필요한 라이브러리 import

from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
import colbert

In [84]:
# 테스트를 위해 전체 문서 중 10000개만 색인 (약 5분 정도 소요됨)

checkpoint = 'colbert-ir/colbertv2.0'
index_name = "test"
doc_maxlen = 300
max_id = 10000

with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=2, kmeans_niters=4) # kmeans_niters specifies the number of iterations of k-means clustering; 4 is a good and fast default.                                                                           # Consider larger numbers for small datasets.

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection[:max_id], overwrite=True)




[Jan 24, 12:20:45] #> Creating directory /content/experiments/notebook/indexes/test 


#> Starting...
#> Joined...


In [85]:
# 검색을 위한 searcher 생성
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name, collection=collection)


[Jan 24, 12:24:52] #> Loading codec...
[Jan 24, 12:24:52] #> Loading IVF...
[Jan 24, 12:24:52] #> Loading doclens...


100%|██████████| 1/1 [00:00<00:00, 453.34it/s]

[Jan 24, 12:24:52] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 12.53it/s]


In [86]:
# 문서 검색 - 영어 문서이기 때문에 영어로 질의 생성
queries = ["What are key factors to consider for minimizing stress when moving an aquarium?"]

for query in queries:
  print(f"#> {query}")

  # Find the top-5 passages for this query
  results = searcher.search(query, k=5)

  # Print out the top-k retrieved passages
  for passage_id, passage_rank, passage_score in zip(*results):
      print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

#> What are key factors to consider for minimizing stress when moving an aquarium?

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . What are key factors to consider for minimizing stress when moving an aquarium?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2024,  3145,  5876,  2000,  5136,  2005,  7163,
         4328,  6774,  6911,  2043,  3048,  2019, 18257,  1029,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

	 [1] 		 21.2 		 When moving an aquarium (from an old location close to the new one) consider the following: Moving the aquarium means stress for inhabitants and you. So, dont place the aquarium at a worse location than before ((more) direct impact of sunlight which might cause algae growning,

## 3. 대화형 IR

준비된 검색엔진과 LLM을 활용하셔 대화형 IR 시스템 구현

In [87]:

persona_qa = """
## Role: 반려 동물 전문가

## Instructions
- 사용자의 이전 메시지 정보 및 주어진 Reference(JSON 형태로 제공) 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 다른 대답 하지 말고 "정보가 부족해서 답을 할 수 없습니다"라고 대답한다.
- 한국어로 답변을 생성한다.
"""

persona_function_calling = """
## Role: 반려 동물 전문가

## Instruction
- 사용자가 대화를 통해 반려 동물 관련된 지식이나 도움을 요청하면 search api를 호출할 수 있어야 한다.
- Search api에 필요한 standalone_query는 영어로 생성한다.
- 반려 동물과 관련되지 않은 나머지 대화 메시지에는 적절한 대답을 생성한다.
"""

tools = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "search knowledge from the user messages sent and received.",
            "parameters": {
                "properties": {
                    "standalone_query": {
                        "type": "string",
                        "description": "English query suitable for use in search from the user messages history."
                    }
                },
                "required": ["standalone_query"],
                "type": "object"
            }
        }
    },
]


In [88]:
# 대화형 Agent를 위한 dialog manager 구현

import json

def conversation_search(messages, persona, tools):
    msg = [{"role": "system", "content": persona}] + messages
    result = client.chat.completions.create(
        model=llm_model,
        messages=msg,
        tools=tools,
        temperature=0,
        seed=1
    )
    response_message = result.choices[0].message

    if response_message.tool_calls:
        for tool_call in response_message.tool_calls:
            function_args = json.loads(tool_call.function.arguments)
            standalone_query = function_args.get("standalone_query")
            results = searcher.search(standalone_query, k=3)
            print("검색어>>: ", standalone_query)
            print()
            print("검색결과>>: ")
            retrieved_context = []
            for passage_id, passage_rank, passage_score in zip(*results):
                retrieved_context.append(searcher.collection[passage_id])
                print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

            msg = [{"role": "system", "content": persona_qa}] + messages
            msg.append({"role": "user", "content": json.dumps(retrieved_context)})
            qaresult = client.chat.completions.create(
                    model=llm_model,
                    messages=msg,
                    temperature=0,
                    seed=1
                )
            print("LLM 대답>>:")
            print(qaresult.choices[0].message.content)
    else:
        print("LLM 대답>>:")
        print(response_message.content)

In [89]:
# OpenAI 라이브러리 및 필요한 모듈을 가져옵니다.
import os
import json
from openai import OpenAI

# OpenAI API 키를 환경변수에서 설정합니다.
os.environ["OPENAI_API_KEY"] = ""

client = OpenAI()
# 사용할 모델을 설정합니다. 여기서는 gpt-3.5-turbo-1106 모델을 사용합니다.
#llm_model = "gpt-4-1106-preview"
llm_model = "gpt-3.5-turbo-1106"


In [91]:

messages = [
{"role": "user", "content": "안녕하세요, 저는 새로운 개 주인이고 방금 영국 마스티프를 입양했어요. 그를 돌보는 데 대해 몇 가지 팁을 줄 수 있나요?"},
{"role": "assistant", "content": "물론이죠! 영국 마스티프 같은 대형견을 돌보는 것은 매우 보람찬 일입니다. 그들은 균형 잡힌 식단, 규칙적인 운동, 그리고 정기적인 수의사 검진이 필요합니다. 그들의 돌봄에 대해 구체적으로 궁금한 점이 있나요?"},
{"role": "user", "content": "네, 특히 그의 식단에 대해 걱정이 되네요. 1살 영국 마스티프에게 얼마나 많은 양을 먹여야 하나요?"},
]

conversation_search(messages, persona_function_calling, tools)


검색어>>:  How much should I feed a 1-year-old English Mastiff?

검색결과>>: 
	 [1] 		 21.5 		 I have a 2 1/2 year old bull mastiff. I have been feeding him Blue Buffalo since I got him at 8 weeks old. He is very lean and active for a bull mastiff. I feed him about 3-4 cups twice a day which averages about 130.00 a month. It is very important that you can afford this breed. I just had to take mine to the vet because he developed some sort of allergies on his skin, eyes and ears and the vet bill was $210.00 with all his medication. This wasnt an option I had to take him an get all his meds or he would have gotten worse. Theyre just like your children, you can expect things to come up and you need to be able to care for them.
	 [2] 		 18.7 		 I breed mastiffs so I will try to help you with this: Age Amount 4-8 weeks 3-4 cups per day spread between 3-4 meals 8-12 weeks 4-6 cups per day spread between 3-4 meals 12-16 weeks 6-8 cups per day spread between 3-4 meals 4 to 6 months 8-10 cups per day 

In [92]:

messages = [
{"role": "user", "content": "저는 고양이를 입양하고 싶은데, 최근에 청각장애가 있는 고양이를 만났어요. 조금 걱정이 되네요. 특별히 고려해야 할 점이 있나요?"},
{"role": "assistant", "content": "청각장애가 있는 고양이를 입양하는 것은 독특하고 보람찬 경험이 될 수 있습니다. 이들은 일반적으로 안전한 실내 환경과 명확한 시각적 의사소통이 필요합니다. 특별히 걱정되는 점이 있나요?"},
{"role": "user", "content": "네, 그들의 행동에 대해 궁금해요. 청각장애가 있는 고양이는 다른 고양이들보다 더 공격적인가요?"}
]

conversation_search(messages, persona_function_calling, tools)


검색어>>:  Are deaf cats more aggressive than other cats?

검색결과>>: 
	 [1] 		 26.2 		 We had a deaf cat, sadly passed away only about a month ago, and to be honest we didnt handle her any differently than our other cat. The things to bear in mind are: Theyre easier to surprise and every cat might react a little differently if startled, particularly from sleep. Ours just kind of jumped up, but others might be a bit more aggressive. They wont react to shouts or loud noises. We used to clap our hands to get the attention of our cats when they were doing things like clawing furniture. That would usually get them to stop, but that was totally useless with her once she went deaf. At that point wed just pick her up and remove her from the area, which seemed to work. Deaf cats still like attention, grooming, and petting. I found that she became even more of a lap cat once she became deaf, possibly because of getting touch feedback from us. In any event, cats are pretty adaptable and deafness will 

In [93]:

messages = [
{"role": "user", "content": "낙타를 키우고 싶은데 어떤 준비가 필요한가요?"}
]

conversation_search(messages, persona_function_calling, tools)

검색어>>:  what do I need to prepare for raising a camel?

검색결과>>: 
	 [1] 		 14.7 		 We raised 200 pigs per year, when they were about 3 to 5 weeks old, we vaccinated, castrated, and nose-ringed. We set up a line - about 3 catchers, the vet with syringes/bottles, the castrating board, and the ringer. A catcher would grab a pig, go thru a gate, get in line & hold the pig while the vet gave the shots. He then moved to the castrate line and held the pig during castration. And then moved to the ringer line - and then put the pig thru a pasture gate. As for cruelty, Im guessing that the ringing was the least cruel of the 3 processes. Hard to tell, pigs scream continuously the entire time that you handle them (be sure to wear ear protection). And wear leather gloves, they reach around and bite your thumb at every opportunity. If you grow pastured hogs, youll need rings, 250 pound hogs can dig under most fences, actually pull posts out of ground, wreck the side of wooden buildings, etc. If you a

In [94]:
# 지식에 대한 질문이 아닌 경우

messages = [
{"role": "user", "content": "저는 고양이를 입양하고 싶은데, 최근에 청각장애가 있는 고양이를 만났어요. 조금 걱정이 되네요. 특별히 고려해야 할 점이 있나요?"},
{"role": "assistant", "content": "청각장애가 있는 고양이를 입양하는 것은 독특하고 보람찬 경험이 될 수 있습니다. 이들은 일반적으로 안전한 실내 환경과 명확한 시각적 의사소통이 필요합니다. 특별히 걱정되는 점이 있나요?"},
{"role": "user", "content": "그런데 너는 누구니?"}
]

#conversation_search(messages, persona_function_calling, tools, {"type": "function", "function": {"name": "searcher"}})
conversation_search(messages, persona_function_calling, tools)

LLM 대답>>:
제가 여기 있어요! 반려 동물 전문가로 여러분이 고양이를 입양하는 데 도움을 드릴 수 있어요. 무엇이든 물어보세요!


#Reference

## Required Package

openai==1.7.2 <br>
colbert-ir==0.2.14 <br>
torch==1.13.1 <br>
faiss-gpu==1.7.0

